# Project Title: Identify customer segments for online retail with the use of K-means clustering

### Purpose/Problem:
E-commerce businesses like any other business depend on Customer Relationship Management software to manage customer relationships and drive customer loyalty and retention. Understanding the behavior of their customers and dividing them into segments as per their spending habits, and frequency of platform’s use - helps an e-commerce platform sharpen its customer success policy and increase revenue coming from repeat customers.

**Dataset Excel: Online Retail**


### Task 1. Data Preparation
1. Load the Data:
Read data from various sources such as CSV files, Excel files, and SQL databases.
2. Understand the Data:
Explore basic information about the data, including head, info, and describe.
Check for missing values in the dataset.
3. Clean the Data:
Handle missing values by either dropping or filling them.
Remove duplicate entries in the dataset.
Correct data types as necessary.
4. Transform the Data:
Normalize or standardize the data.
Encode categorical variables.
5. Feature Engineering:
Create new features based on existing ones.
Perform feature selection to identify the most relevant features.

#  Import Required Libraries

In [10]:
#basic libraries for linear algebra and data procesing
import numpy as np
import pandas as pd
import missingno as msno
from datetime import datetime, timedelta

#visualization
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from yellowbrick.cluster import KElbowVisualizer

#nlp
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

#modeling
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#time and warnings
import time
%matplotlib inline
sns.set_context('poster', font_scale=0.5)

In [12]:
import warnings
# Suppressing all warnings
warnings.filterwarnings("ignore")


# 1. Loading the Dataset

In [15]:
df = pd.read_excel('C:/Users/DELL/anaconda_projects/Snap Deal Project/Online Retail.xlsx')

In [16]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


# 2. Data Understanding

In [18]:
# Get the number of rows and columns
num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 541909
Number of columns: 8


In [19]:
# Get first 10 rows
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [20]:
# Display the last 10 rows of the dataset
df.tail(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541899,581587,22726,ALARM CLOCK BAKELIKE GREEN,4,2011-12-09 12:50:00,3.75,12680.0,France
541900,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680.0,France
541901,581587,22367,CHILDRENS APRON SPACEBOY DESIGN,8,2011-12-09 12:50:00,1.95,12680.0,France
541902,581587,22629,SPACEBOY LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France
541903,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680.0,France
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [21]:
# Display dataset summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [22]:
# Display column names
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [23]:
# Count the occurrences of each unique InvoiceNo in the dataset
df['InvoiceNo'].value_counts()

InvoiceNo
573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554023        1
554022        1
554021        1
554020        1
C558901       1
Name: count, Length: 25900, dtype: int64

In [24]:
# Count occurrences of each product (StockCode) in transactions
df['StockCode'].value_counts()

StockCode
85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: count, Length: 4070, dtype: int64

In [25]:
# Count purchases made by each unique customer
df['CustomerID'].value_counts()

CustomerID
17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
15070.0       1
15753.0       1
17065.0       1
16881.0       1
16995.0       1
Name: count, Length: 4372, dtype: int64

In [26]:
# Count the number of transactions from each country
df['Country'].value_counts()

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


### Check for missing values in the dataset.

In [28]:
total_missing = df.isnull().values.sum()

# Calculate the percentage of missing values per feature
missing_percentage = df.isnull().sum() * 100 / len(df)

print(f'Total number of missing values: {total_missing}')
print('Missing values per feature (%):')
print(missing_percentage)

Total number of missing values: 136534
Missing values per feature (%):
InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64


In [29]:
# Visualizing missing data in the dataframe
msno.matrix(df)

<Axes: >

## 3. Data Cleaninig

### Removing the missing values

Nearly 25% of the CustomerID values are missing, and this column is crucial for customer-level analysis, I will remove all rows with missing CustomerID values. Although the missing data for Description is relatively small (0.27%), the absence of customer information is more impactful, and retaining incomplete customer data could affect the quality of the analysis.

In [32]:
# Removing rows with missing values
df.dropna(inplace=True)

In [33]:
# Resetting the index after dropping rows
df.reset_index(drop=True, inplace=True)

In [34]:
# Display the first 10 rows of the cleaned DataFrame
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [35]:
# Display the total number of missing values and the length of dataset after cleaning
print('Total Number of missing values after: ', df.isnull().values.sum())
print('Data length: ', len(df))

Total Number of missing values after:  0
Data length:  406829


### After removing null value from our dataset.

Now we have **406829 rows** and **8 columns** in our dataset.

### Checking for duplicate rows

Duplicates in this case refer to instances where the same customer purchases different products.

In [38]:
# Checking for duplicate rows in the DataFrame
duplicates_df = df.duplicated()
print(duplicates_df.sum())

5225


In [39]:
# Extracting the first duplicate row from the DataFrame
duplicates_row1 = df[duplicates_df == True].iloc[0]
print(duplicates_row1)

InvoiceNo                           536409
StockCode                            21866
Description    UNION JACK FLAG LUGGAGE TAG
Quantity                                 1
InvoiceDate            2010-12-01 11:45:00
UnitPrice                             1.25
CustomerID                         17908.0
Country                     United Kingdom
Name: 517, dtype: object


In [40]:
# Extracting the second duplicate row from the DataFrame
duplicates_row2 = df[duplicates_df == True].iloc[1]
print(duplicates_row2)

InvoiceNo                             536409
StockCode                              22866
Description    HAND WARMER SCOTTY DOG DESIGN
Quantity                                   1
InvoiceDate              2010-12-01 11:45:00
UnitPrice                                2.1
CustomerID                           17908.0
Country                       United Kingdom
Name: 527, dtype: object


In [41]:
# Displaying all duplicate rows in the DataFrame
duplicates_all = df[df.duplicated()]
duplicates_all

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom
...,...,...,...,...,...,...,...,...
406595,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
406609,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
406612,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom
406619,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.0,United Kingdom


In [42]:
# Convert CustomerID to integer (as it's currently float but represents an ID)
df['CustomerID'] = df['CustomerID'].astype('Int64')

# Convert InvoiceNo and StockCode to string (if they should be treated as categorical/text values)
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)

# Convert UnitPrice to a more precise numeric type (optional)
df['UnitPrice'] = df['UnitPrice'].astype(float)

# Ensure InvoiceDate is in datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Verify changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      406829 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 25.2+ MB


In [43]:
df.drop(columns=['InvoiceDate']).describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.0
mean,12.061303,3.460471,15287.69057
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.0
25%,2.000000,1.250000,13953.0
50%,5.000000,1.950000,15152.0
75%,12.000000,3.750000,16791.0
max,80995.000000,38970.000000,18287.0


In [44]:
df[df['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
406377,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
406461,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
406635,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
406636,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [45]:
df[df['Quantity'] > 1000]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
3645,536809,84950,ASSORTED COLOUR T-LIGHT HOLDER,1824,2010-12-02 16:48:00,0.55,15299,United Kingdom
3740,536830,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,2880,2010-12-02 17:38:00,0.18,16754,United Kingdom
3741,536830,21915,RED HARMONICA IN BOX,1400,2010-12-02 17:38:00,1.06,16754,United Kingdom
4286,536890,17084R,ASSORTED INCENSE PACK,1440,2010-12-03 11:48:00,0.16,14156,EIRE
10946,537659,22189,CREAM HEART CARD HOLDER,1008,2010-12-07 16:43:00,2.31,18102,United Kingdom
...,...,...,...,...,...,...,...,...
402106,581115,22413,METAL SIGN TAKE IT OR LEAVE IT,1404,2011-12-07 12:20:00,2.75,15195,United Kingdom
403246,581175,23084,RABBIT NIGHT LIGHT,1440,2011-12-07 15:16:00,1.79,14646,Netherlands
405998,581458,22197,POPCORN HOLDER,1500,2011-12-08 18:45:00,0.72,17949,United Kingdom
405999,581459,22197,POPCORN HOLDER,1200,2011-12-08 18:46:00,0.72,17949,United Kingdom


In [46]:
df[df['UnitPrice'] > 1000]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
28939,C540271,M,Manual,-1,2011-01-06 11:51:00,1126.00,12503,Spain
45380,C542348,M,Manual,-1,2011-01-27 12:09:00,1715.85,12539,Spain
78283,C546325,M,Manual,-1,2011-03-11 10:15:00,1687.17,14911,EIRE
78284,C546327,M,Manual,-1,2011-03-11 10:18:00,1687.17,14911,EIRE
78285,546328,M,Manual,1,2011-03-11 10:19:00,1687.17,14911,EIRE
78286,546329,M,Manual,1,2011-03-11 10:22:00,1687.17,14911,EIRE
99563,548813,M,Manual,1,2011-04-04 13:03:00,2382.92,12744,Singapore
99565,548813,M,Manual,1,2011-04-04 13:03:00,1252.95,12744,Singapore
99566,548820,M,Manual,1,2011-04-04 13:04:00,2053.07,12744,Singapore
99568,C548830,M,Manual,-1,2011-04-04 13:08:00,2382.92,12744,Singapore


In [47]:
# List of specific descriptions to count
descriptions = ['Manual', 'Discount', 'POSTAGE', 'CRUK Commission', 'DOTCOM POSTAGE']

# Count occurrences for each description and store in a dictionary
desc_counts = {desc: (df['Description'] == desc).sum() for desc in descriptions}

for key, value in desc_counts.items():
    print(f"Sum of value {key}: {value}")


Sum of value Manual: 465
Sum of value Discount: 77
Sum of value POSTAGE: 1196
Sum of value CRUK Commission: 16
Sum of value DOTCOM POSTAGE: 16


In [48]:
# Count the number of StockCode values shorter than 5 characters
count_stockcode_less_5 = (df['StockCode'].str.len() < 5).sum()

print(f"Count of values with length less than 5 in StockCode: {count_stockcode_less_5}")

Count of values with length less than 5 in StockCode: 1908


In [49]:
# Count occurrences of each unique StockCode with length < 5
stockcode_counts = code_less_5['StockCode'].value_counts()
print(stockcode_counts)

NameError: name 'code_less_5' is not defined

### Removing unwanted values as per conclusion above

In [ ]:
# Removing unwanted StockCode values
df = df[~df['StockCode'].isin(['POST', 'M', 'C2', 'D', 'CRUK', 'DOT', 'PADS'])]

In [ ]:
df.drop(columns=['InvoiceDate']).describe()

In [ ]:
#checking the unit price featureisplays rows where the UnitPrice is greater than 1000.
df[df['UnitPrice'] > 1000]

In [ ]:
description_counts = df['Description'].value_counts()

# Print the counts of each specific value
print(f"Sum of value manual: {description_counts.get('Manual', 0)}")
print(f"Sum of value discount: {description_counts.get('Discount', 0)}")
print(f"Sum of value postage: {description_counts.get('POSTAGE', 0)}")
print(f"Sum of value cruk commission: {description_counts.get('CRUK Commission', 0)}")
print(f"Sum of value dotcom postage: {description_counts.get('DOTCOM POSTAGE', 0)}")


In [ ]:
#checking the huge negative value in Quantity
df[df['Quantity'] < -5000]

In [ ]:
#checking the huge positive value in Quantity
df[df['Quantity'] > 5000]

### Conclusion:

After addressing the removal of unwanted values in specific features, an issue arose during the purchase of certain items, such as PAPER CRAFT and LITTLE BIRDIE. I suspect that the customer either made an error while selecting a quantity of approximately 81K or changed their mind and returned the product. Both the purchased and returned quantities remain in the dataset. As the purchase quantity is recorded as a positive value, the total will result in zero during feature engineering. Therefore, I decided to retain these outliers in the data for now, unless further analysis suggests they should be removed if my assumptions turn out to be incorrect.

## 4. Transform the Data:
Standardize the data.
Encode categorical variables.


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# Standardizing (mean=0, std=1) for features with continuous values
scaler = StandardScaler()
df[['Quantity', 'UnitPrice']] = scaler.fit_transform(df[['Quantity', 'UnitPrice']])

## 5. Feature Engineering:

## a. RFM Analysis

In [ ]:
df.head()

In [ ]:
#converting the date feature to a datetime object
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df.info()

### Recency

In [ ]:
#setting the reference date to the end date of the analysis
reference_date = df['InvoiceDate'].max()

#grouping data by customer and getting the latest purchase date for each customer
grouped = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()

#calculatingthe recency for each customer by subtracting the most recent purchase date from the reference date
grouped['Recency'] = (reference_date - grouped['InvoiceDate']).dt.days

#merging the recency value to the dataframe
df = df.merge(grouped[['CustomerID', 'Recency']], on='CustomerID', how='left')

### Frequency

In [ ]:
#calculating the frequency for each customer by counting the number of purchases
frequency = df.groupby('CustomerID')['InvoiceNo'].count()

#merging the frequency value to the dataframe
df = df.merge(frequency, on='CustomerID', how='left')
df.rename(columns={'InvoiceNo_x':'InvoiceNo', 'InvoiceNo_y':'Frequency'}, inplace=True)

### Monetary Value

In [ ]:
#calculating the monetary value for each customer 
monetary_value = df.groupby('CustomerID')['UnitPrice'].sum()

#merging the monetary value to the dataframe
df = df.merge(monetary_value, on='CustomerID', how='left')
df.rename(columns={'UnitPrice_x':'UnitPrice', 'UnitPrice_y':'MonetaryValue'}, inplace=True)

### RFM Segmentation using scores

The customers are grouped into different segments based on their RFM level, and calculate the mean values for recency, frequency, and monetary value for each segment. 
The result I get is a summary table that provides us with valuable insights into the behavior of each segment of customers.

In [ ]:
#normalizing the values
r_quartile = pd.qcut(df['Recency'], 4, labels=[4, 3, 2, 1])
f_quartile = pd.qcut(df['Frequency'], 4, labels=[1, 2, 3, 4])
m_quartile = pd.qcut(df['MonetaryValue'], 4, labels=[1, 2, 3, 4])
df['RFM_Score'] = r_quartile.astype(int) + f_quartile.astype(int) + m_quartile.astype(int)

In [ ]:
#creating the rfm_level function
def assign_rfm_level(row):
    """
    
    This function assigns the RFM level based on the RFM score.
    :parameter: RFM scores from the dataframe
    :return: The RFM level category as string
    
    """

    if row['RFM_Score'] >= 10:
        return 'Best Customers'
    elif (row['RFM_Score'] < 10) and (row['RFM_Score'] >= 6):
        return 'Loyal Customers'
    elif (row['RFM_Score'] < 6) and (row['RFM_Score'] >= 4):
        return 'Potential Loyalists'
    elif (row['RFM_Score'] < 4) and (row['RFM_Score'] >= 2):
        return 'Promising'
    else:
        return 'Needs Attention'

df['RFM_Level'] = df.apply(assign_rfm_level, axis=1)

In [ ]:
df.head(10)

## 6. Data Visualization

### Visualizing the RFM Level customers using a bar plot

In [ ]:
#barplot visualzizing the number of customers per rfm level
plt.figure(figsize=(9,3))
sns.countplot(data = retail, x = 'RFM_Level', palette = 'Set1')

plt.xlabel('RFM Level', fontsize = 12)
plt.ylabel('Number of Customers')
plt.title('RFM Levels of Customers')
plt.show()